Webscraping


In [2]:
from IPython.display import display
import json
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re
import requests
from bs4 import BeautifulSoup

In [14]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

chromeOptions = webdriver.ChromeOptions()
#it is way to add options

path = 'C:/Users/User/Documents/Github/Films_DM_2023/chromedriver_win32/' #the path
driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))

driver.get("https://www.allocine.fr/")
time.sleep(1) # Let the user actually see something!

C:\Users\User\AppData\Local\Temp\ipykernel_18608\4193159110.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))


In [15]:
#Accepter les cookies pour pouvoir passer à la suite
cookies = driver.find_element(By.CLASS_NAME, "jad_cmp_paywall_button-cookies").click()
time.sleep(2)

#Aller dans films 
films = driver.find_element()
